In [1]:
import os
import pandas as pd
from imageio import imread
import math
import numpy as np
import cv2
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.svm import SVC,LinearSVC
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import classification_report
import pickle

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline

In [2]:
def load_data(filedir, filepath, csvfile):
    data = np.load(os.path.join(filedir, filepath), allow_pickle=True)/255
    train_info = pd.read_csv(os.path.join(filedir, csvfile))
    labels = np.array(train_info['ClassId'])
    
    return data, labels

def randomize_data(data, labels):
    randomize = np.arange(len(labels))
    np.random.shuffle(randomize)
    X = data[randomize]
    y = labels[randomize]
    
    return X,y

def train_model(trainData, trainLabels, modelName):
    if os.path.isfile(modelName):
        print("[INFO] loading classifier: SVC trained on no feature extraction...")
        svc = pickle.load(open(modelName, 'rb'))
        print("[INFO] Classifer is loaded as instance ::svc::")
    else:
        print("[INFO] pre-trained classifier not found. \n Training Classifier SVC")
        svc = LinearSVC()
        svc.fit(trainData,trainLabels)
        print("[INFO] Succefully trained the classsifier. \n Saving the classifier for further use")
        pickle.dump(svc, open(modelName, 'wb')) 
        print("[INFO] Classifier Saved")
    
    return svc

In [3]:
data_dir = './'
trainData, trainLabels = load_data(data_dir, './SIFT/trainProcessedSIFT.npy', 'Train.csv')
testData, testLabels = load_data(data_dir, './SIFT/testProcessedSIFT.npy', 'Test.csv')

In [4]:
trainData, trainLabels = randomize_data(trainData, trainLabels)

In [31]:
svc = train_model(trainData.reshape(len(trainLabels), -1), trainLabels, 'svm_SIFT_processed.pkl')

[INFO] loading classifier: SVC trained on no feature extraction...
[INFO] Classifer is loaded as instance ::svc::


In [32]:
predictions = svc.predict(testData.reshape(len(testLabels), -1))
 
# show a final classification report demonstrating the accuracy of the classifier
print("EVALUATION ON TESTING DATA")
print(classification_report(testLabels, predictions))

EVALUATION ON TESTING DATA
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        60
           1       0.36      0.58      0.45       720
           2       0.43      0.55      0.49       750
           3       0.31      0.05      0.09       450
           4       0.53      0.57      0.55       660
           5       0.33      0.35      0.34       630
           6       0.94      0.10      0.18       150
           7       0.61      0.40      0.48       450
           8       0.79      0.31      0.45       450
           9       0.62      0.49      0.55       480
          10       0.37      0.75      0.49       660
          11       0.36      0.39      0.37       420
          12       0.48      0.85      0.61       690
          13       0.35      0.87      0.50       720
          14       0.64      0.53      0.58       270
          15       1.00      0.05      0.09       210
          16       0.00      0.00      0.00       150


c:\users\nhinp3\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
estimators = [1,10, 50, 100, 500]
def train_model(estimators, trainData, trainLabels, testData, testLabels):
    for n in estimators:
        print(f'Linear SVM WITH C={n}')
        if os.path.isfile(f"svm_{str(n)}_processed_sift.pkl"):
            print("[INFO] loading classifier: SVM trained on ori images...")
            svm = pickle.load(open(f"svm_{str(n)}_processed_sift.pkl", 'rb'))
            print("[INFO] Classifer is loaded as instance ::rf::")
        else:
            print("[INFO] pre-trained classifier not found. \n Training Classifier SVM")
            svm = LinearSVC(C=n, class_weight='balanced', max_iter=max_iter)
            svm.fit(trainData.reshape(len(trainLabels), -1),trainLabels)
            print("[INFO] Succefully trained the classsifier. \n Saving the classifier for further use")
            pickle.dump(svm, open(f"svm_{str(n)}_processed_sift.pkl", 'wb')) 
            print("[INFO] Classifier Saved")
            
        predictions = svm.predict(testData.reshape(len(testLabels), -1))
 
        # show a final classification report demonstrating the accuracy of the classifier
        print("EVALUATION ON TESTING DATA FOR" + str(n) + 'SVM')
        print(classification_report(testLabels, predictions))
        
        print("ACCURACY ON TRAINING DATA")
        print(svm.score(trainData.reshape(len(trainLabels), -1), trainLabels))
        print('-------------------------------------------------------')

In [20]:
train_model(estimators, trainData, trainLabels, testData, testLabels)

Linear SVM WITH C=1
[INFO] loading classifier: SVM trained on ori images...
[INFO] Classifer is loaded as instance ::rf::
EVALUATION ON TESTING DATA FOR1SVM
              precision    recall  f1-score   support

           0       0.06      0.23      0.09        60
           1       0.60      0.44      0.51       720
           2       0.69      0.35      0.47       750
           3       0.63      0.21      0.31       450
           4       0.59      0.41      0.48       660
           5       0.47      0.28      0.35       630
           6       0.31      0.75      0.44       150
           7       0.56      0.45      0.50       450
           8       0.57      0.35      0.43       450
           9       0.73      0.53      0.61       480
          10       0.74      0.30      0.43       660
          11       0.49      0.25      0.33       420
          12       0.70      0.78      0.73       690
          13       0.62      0.68      0.65       720
          14       0.36      0.8

0.5480884490805682
-------------------------------------------------------
Linear SVM WITH C=100
[INFO] loading classifier: SVM trained on ori images...
[INFO] Classifer is loaded as instance ::rf::
EVALUATION ON TESTING DATA FOR100SVM
              precision    recall  f1-score   support

           0       0.18      0.37      0.24        60
           1       0.71      0.53      0.61       720
           2       0.71      0.51      0.60       750
           3       0.59      0.36      0.45       450
           4       0.64      0.62      0.63       660
           5       0.57      0.47      0.51       630
           6       0.60      0.74      0.66       150
           7       0.53      0.59      0.56       450
           8       0.54      0.48      0.51       450
           9       0.70      0.68      0.69       480
          10       0.70      0.53      0.60       660
          11       0.47      0.34      0.39       420
          12       0.81      0.77      0.79       690
       